In [1]:
# Import Dependencies
import json
import requests
import gmaps
from config import api_key
from config import gkey
from pprint import pprint
import pandas as pd
from datetime import date
from datetime import timedelta
import requests
import urllib
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import geopy.distance as gp
import warnings
warnings.filterwarnings('ignore')

# Food Deserts

A Food Desert is an area that has limited access to affordable and nutritious food. Food deserts tend to be inhabited by low-income residents with reduced mobility; this makes them a less attractive market for large supermarket chains. Food deserts lack suppliers of fresh foods, such as meats, fruits, and vegetables. Instead, the available foods are often processed and high in sugar and fats, which are known contributors to the proliferation of obesity in the U.S.

### Beginning the data-gathering process

We found that the Percentage of Free or Reduced Lunches (Percent_FRL) at school is a strong indicator of poverty in an area:
https://nces.ed.gov/blogs/nces/post/free-or-reduced-price-lunch-a-proxy-for-poverty

Using this metric, we identified 90 schools across 7 counties of Georgia with Percent_FRL of >= 92%. We collected these school names, physical addresses and the their Percent_FRL in a CSV file.

### Approach
1. Get the name and addresses of all the schools which have the >90% FRL (free-reduced lunch) (%FRL is used as an indicator of poverty).
2. Find out the number of grocery stores within 5-10 miles of these schools (usually people live near the schools and higher percentage FRL means that the neighborhood is poor).
3. Create a Heat Map these grocery stores.
4. Find out any Fast Food joints / Dollar Stores within 5 miles of these locations. Usually in Food Deserts there is proliferation of these establishments selling processed foods.
5. Creat a Heat Map these Fast Food places / Dollar Stores.

In [2]:
infile = pd.read_csv("../Datasets/GA_Counties_Schools.csv")
school_df = pd.DataFrame(infile)
school_df.head()

,County_ID,County_Name,School_ID,School_Name,Percent_FRL,Address,City,State,ZipCode
0,605,Baldwin County,189,Baldwin High School,92.86,155 Ga Highway 49 W,Milledgeville,GA,31061
1,605,Baldwin County,300,Lakeview Academy,92.78,220 N Abc Dr,Milledgeville,GA,31061
2,605,Baldwin County,302,Lakeview Primary,92.92,372 Blandy Rd NW,Milledgeville,GA,31061
3,605,Baldwin County,303,Midway Hills Academy,92.90,101 Carl Vinson Rd SE,Milledgeville,GA,31061
4,605,Baldwin County,301,Midway Hills Primary,92.86,375 Blandy Rd NW,Milledgeville,GA,31061


## Getting Coordinates of the Schools

In [3]:
school_coordf = pd.DataFrame(columns=['School_ID', 'County_Name', 'School_Name', 'Latitude', 'Longitude'])
                                   
errors = []

for row in school_df.itertuples():
    County_Name = row.County_Name
    School_ID = row.School_ID
    School_Name = row.School_Name
    Zip = str(row.ZipCode)
    school_addr = row.Address+','+row.City+','+row.State+','+Zip
    api_response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(school_addr, gkey))
    api_response_dict = api_response.json()

    try:
        if api_response_dict['status'] == 'OK':
            Latitude = api_response_dict['results'][0]['geometry']['location']['lat']
            Longitude = api_response_dict['results'][0]['geometry']['location']['lng']

            current_record = [School_ID, County_Name, School_Name, Latitude, Longitude]
            school_coordf.loc[len(school_coordf)] = current_record
   
    except KeyError:
            errors.append('Key Error')

school_coordf



,School_ID,County_Name,School_Name,Latitude,Longitude
0,189,Baldwin County,Baldwin High School,33.072068,-83.258449
1,300,Baldwin County,Lakeview Academy,33.090509,-83.264271
2,302,Baldwin County,Lakeview Primary,33.080177,-83.268580
3,303,Baldwin County,Midway Hills Academy,33.019280,-83.227823
4,301,Baldwin County,Midway Hills Primary,33.079949,-83.271429
...,...,...,...,...,...
85,1052,Lowndes County,W.G. Nunn Elementary,30.859155,-83.263312
86,205,Wilkes County,Washington-Wilkes Elementary School,33.734583,-82.725189
87,105,Wilkes County,Washington-Wilkes Primary School,33.728877,-82.720109
88,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498


In [4]:
school_df = school_df.merge(school_coordf[['School_ID','Latitude', 'Longitude']], on = 'School_ID', how = 'left')
school_df

,County_ID,County_Name,School_ID,School_Name,Percent_FRL,Address,City,State,ZipCode,Latitude,Longitude
0,605,Baldwin County,189,Baldwin High School,92.86,155 Ga Highway 49 W,Milledgeville,GA,31061,33.072068,-83.258449
1,605,Baldwin County,300,Lakeview Academy,92.78,220 N Abc Dr,Milledgeville,GA,31061,33.090509,-83.264271
2,605,Baldwin County,302,Lakeview Primary,92.92,372 Blandy Rd NW,Milledgeville,GA,31061,33.080177,-83.268580
3,605,Baldwin County,303,Midway Hills Academy,92.90,101 Carl Vinson Rd SE,Milledgeville,GA,31061,33.019280,-83.227823
4,605,Baldwin County,301,Midway Hills Primary,92.86,375 Blandy Rd NW,Milledgeville,GA,31061,33.079949,-83.271429
...,...,...,...,...,...,...,...,...,...,...,...
111,757,Wilkes County,205,Washington-Wilkes Elementary School,94.09,109 East St,Washington,GA,30673,33.734583,-82.725189
112,757,Wilkes County,105,Washington-Wilkes Primary School,94.16,910 E Robert Toombs Ave,Washington,GA,30673,33.593735,-84.318128
113,757,Wilkes County,105,Washington-Wilkes Primary School,94.16,910 E Robert Toombs Ave,Washington,GA,30673,33.728877,-82.720109
114,757,Wilkes County,210,Washington-Wilkes Middle School,94.22,1180 Tignall Rd,Washington,GA,30673,33.760145,-82.732498


In [5]:
stores = []
for row in school_df.itertuples():
    
    County_Name = row.County_Name
    School_ID = row.School_ID
    School_Name = row.School_Name
      
    School_Lat = row.Latitude
    School_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{School_Lat},{School_Long}", 
          "radius": 10000, 
          "type": "supermarket"}

    data = requests.get(base_url, params = params).json()
    
    try:
        if data['status'] == 'OK':
            i = 0
            for row in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                
                current_record = [School_ID, County_Name, School_Name, School_Lat, School_Long,\
                                    Store_Name, Store_Lat, Store_Long, Store_Address, Store_PlaceID]
                stores.append(current_record)
                i = i + 1
                
    except IndexError:
        errors.append('Index Out of Range')    
    


store_df = pd.DataFrame(stores, columns=['School_ID', 'County_Name', 'School_Name', 'School_Lat', 'School_Long', 'Store_Name',\
            'Store_Lat','Store_Long', 'Store_Address','Store_PlaceID'])
store_df




,School_ID,County_Name,School_Name,School_Lat,School_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
1,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
2,189,Baldwin County,Baldwin High School,33.072068,-83.258449,Family Dollar,33.055031,-83.239311,"355 Allen Memorial Drive, Milledgeville",ChIJpUDdw1vJ9ogReo5URShyfMA
3,300,Baldwin County,Lakeview Academy,33.090509,-83.264271,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
4,300,Baldwin County,Lakeview Academy,33.090509,-83.264271,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
...,...,...,...,...,...,...,...,...,...,...
392,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
393,210,Wilkes County,Washington-Wilkes Middle School,33.760145,-82.732498,Family Dollar,33.725677,-82.718119,"164 Bi-Lo Plaza, Washington",ChIJlYwf8G2l94gRxbbIZSoRgvw
394,110,Wilkes County,Washington-Wilkes Comprehensive High School,33.760145,-82.732498,Terry's IGA,33.736149,-82.739447,"115 North Spring Street, Washington",ChIJidCGaIuv94gR-pjx2WR5QAY
395,110,Wilkes County,Washington-Wilkes Comprehensive High School,33.760145,-82.732498,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM


In [7]:
only_stores = []
for row in school_df.itertuples():
    
    County_Name = row.County_Name   
    School_Lat = row.Latitude
    School_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{School_Lat},{School_Long}", 
          "radius": 10000, 
          "type": "supermarket"}

    data = requests.get(base_url, params = params).json()
    
    try:
        if data['status'] == 'OK':
            i = 0
            for row in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                
                current_record = [County_Name, Store_Name, Store_Lat, Store_Long, Store_Address, Store_PlaceID]
                only_stores.append(current_record)
                i = i + 1
                
    except IndexError:
        errors.append('Index Out of Range')    
    


only_stores_df = pd.DataFrame(only_stores, columns=['County_Name', 'Store_Name','Store_Lat','Store_Long',\
                                                    'Store_Address','Store_PlaceID'])
only_stores_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,Baldwin County,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
1,Baldwin County,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
2,Baldwin County,Family Dollar,33.055031,-83.239311,"355 Allen Memorial Drive, Milledgeville",ChIJpUDdw1vJ9ogReo5URShyfMA
3,Baldwin County,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
4,Baldwin County,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
...,...,...,...,...,...,...
392,Wilkes County,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
393,Wilkes County,Family Dollar,33.725677,-82.718119,"164 Bi-Lo Plaza, Washington",ChIJlYwf8G2l94gRxbbIZSoRgvw
394,Wilkes County,Terry's IGA,33.736149,-82.739447,"115 North Spring Street, Washington",ChIJidCGaIuv94gR-pjx2WR5QAY
395,Wilkes County,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM


In [10]:
only_stores_df.drop_duplicates(inplace=True)
only_stores_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,Baldwin County,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
1,Baldwin County,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
2,Baldwin County,Family Dollar,33.055031,-83.239311,"355 Allen Memorial Drive, Milledgeville",ChIJpUDdw1vJ9ogReo5URShyfMA
5,Baldwin County,ALDI,33.107343,-83.254181,"1978 North Columbia Street, Milledgeville",ChIJ7e0uBHe49ogRA7mBl9J0k8E
18,Baldwin County,ALDI,33.551046,-84.255041,"5420 North Henry Boulevard, Stockbridge",ChIJ0zKb7T1W9IgR1wzlIeh8LC0
...,...,...,...,...,...,...
382,Wilkes County,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
383,Wilkes County,Family Dollar,33.725677,-82.718119,"164 Bi-Lo Plaza, Washington",ChIJlYwf8G2l94gRxbbIZSoRgvw
384,Wilkes County,H Mart Riverdale,33.582893,-84.412643,"6335 Georgia 85, Riverdale",ChIJTfOT55389IgRMdwExIWG6Ho
385,Wilkes County,ALDI,33.551046,-84.255041,"5420 North Henry Boulevard, Stockbridge",ChIJ0zKb7T1W9IgR1wzlIeh8LC0


In [24]:
grocery_stores_df = only_stores_df.loc[(only_stores_df["Store_Name"].str.contains(r'^(?:(?!Dollar).)*$'))]
grocery_stores_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,Baldwin County,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",ChIJ0S-cSXTI9ogRDi8J4rYg5i8
1,Baldwin County,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",ChIJ792XhnW49ogRzX5zsT1Cjdg
5,Baldwin County,ALDI,33.107343,-83.254181,"1978 North Columbia Street, Milledgeville",ChIJ7e0uBHe49ogRA7mBl9J0k8E
18,Baldwin County,ALDI,33.551046,-84.255041,"5420 North Henry Boulevard, Stockbridge",ChIJ0zKb7T1W9IgR1wzlIeh8LC0
19,Baldwin County,ALDI,33.441894,-84.322878,"11499 Tara Boulevard, Lovejoy",ChIJX7zpyuL29IgRptD3eaYlZ14
...,...,...,...,...,...,...
381,Wilkes County,Terry's IGA,33.736149,-82.739447,"115 North Spring Street, Washington",ChIJidCGaIuv94gR-pjx2WR5QAY
382,Wilkes County,Ingles Markets,33.729814,-82.717518,"110 Ann Denard Drive, Washington",ChIJnWtN3W2l94gRxkhtinYYczM
384,Wilkes County,H Mart Riverdale,33.582893,-84.412643,"6335 Georgia 85, Riverdale",ChIJTfOT55389IgRMdwExIWG6Ho
385,Wilkes County,ALDI,33.551046,-84.255041,"5420 North Henry Boulevard, Stockbridge",ChIJ0zKb7T1W9IgR1wzlIeh8LC0


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
grocery_store_info = [info_box_template.format(**row) for index, row in grocery_stores_df.iterrows()]
locations = grocery_stores_df[["Store_Lat","Store_Long"]]

In [26]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = grocery_store_info)

fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
dollar_stores_df = only_stores_df.loc[(only_stores_df["Store_Name"].str.contains('Dollar'))]
dollar_stores_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
2,Baldwin County,Family Dollar,33.055031,-83.239311,"355 Allen Memorial Drive, Milledgeville",ChIJpUDdw1vJ9ogReo5URShyfMA
25,Clayton County,Family Dollar,33.583420,-84.313688,"2350 Lake Harbin Road, Morrow",ChIJaV_fEtb-9IgRnuo4JeMnKAI
47,Clayton County,Family Dollar,30.836751,-83.317776,"964 Saint Augustine Road, Valdosta",ChIJc4T_1L9m7ogRI7Ht7rcBbC4
48,Clayton County,Family Dollar,30.856150,-83.280243,"1905 North Ashley Street, Valdosta",ChIJyQ6q_whn7ogRlOmWjkS-eQk
50,Clayton County,Family Dollar,33.581123,-84.471901,"6385 Old National Highway, Atlanta",ChIJe6LPrILk9IgR-dySV--a3Ls
116,Clayton County,Family Dollar,30.882300,-83.266300,"3820 Inner Perimeter Road, Valdosta",ChIJawTapexn7ogRWCmZicGu4ag
132,Clayton County,Family Dollar,33.725677,-82.718119,"164 Bi-Lo Plaza, Washington",ChIJlYwf8G2l94gRxbbIZSoRgvw
137,Clayton County,Family Dollar,32.732101,-82.721858,"212 West Trilby Street, Wrightsville",ChIJBQHZVSvO8IgRMe3LlKPs378
156,Clayton County,Family Dollar,33.452406,-84.325361,"11108 Tara Boulevard, Lovejoy",ChIJaxiUzhX39IgRWxySNUd0818
272,Clayton County,Family Dollar,32.172863,-82.593758,"601 South Mason Street, Mount Vernon",ChIJTZX5WgX18IgRGfc859Ze39A


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
dollar_store_info = [info_box_template.format(**row) for index, row in dollar_stores_df.iterrows()]
locations = dollar_stores_df[["Store_Lat","Store_Long"]]

In [28]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = dollar_store_info)

fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
fastfood = []
for row in school_df.itertuples():
    
    County_Name = row.County_Name   
    School_Lat = row.Latitude
    School_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{School_Lat},{School_Long}", 
          "radius": 10000, 
          "type": "restaurant"}

    data = requests.get(base_url, params = params).json()
    
#     pprint(data)
    try:
        if data['status'] == 'OK':
            i = 0
            for row in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                
                current_record = [County_Name, Store_Name, Store_Lat, Store_Long, Store_Address, Store_PlaceID]
                fastfood.append(current_record)
                i = i + 1
                
    except IndexError:
        errors.append('Index Out of Range')    
    


fastfood_df = pd.DataFrame(fastfood, columns=['County_Name', 'Store_Name','Store_Lat','Store_Long',\
                                                    'Store_Address','Store_PlaceID'])
fastfood_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,Baldwin County,Ruby Tuesday,33.110224,-83.256479,"2440 North Columbia Street, Milledgeville",ChIJWzuGB3q49ogR7nEV2FMDl84
1,Baldwin County,Papa John's Pizza,33.105102,-83.253156,"1960 North Columbia Street, Milledgeville",ChIJjy9AQHa49ogRu4xOOjIXP_E
2,Baldwin County,Waffle House,33.099574,-83.246011,"1683 North Columbia Street, Milledgeville",ChIJBV7JQovH9ogRjb4IDNZftGM
3,Baldwin County,Lieu's Peking Restaurant,33.112239,-83.255534,"2485 North Columbia Street STE 101, Milledgeville",ChIJ3_c7KHq49ogREL_631kUmZM
4,Baldwin County,Ruby Tuesday,33.110224,-83.256479,"2440 North Columbia Street, Milledgeville",ChIJWzuGB3q49ogR7nEV2FMDl84
...,...,...,...,...,...,...
437,Wilkes County,The Hot Box,33.736864,-82.740227,"20 West Robert Toombs Avenue, Washington",ChIJPyv5gYyv94gR_k9rrzTwuTg
438,Wilkes County,Shell,33.737671,-82.749277,"16 Lexington Avenue, Washington",ChIJVxfMDZOv94gRYJSaoejrAB0
439,Wilkes County,Wings N Things,33.737777,-82.740444,"111 North Allison Street, Washington",ChIJnWhm-Yqv94gRMZN0rxVpLBc
440,Wilkes County,The Hot Box,33.736864,-82.740227,"20 West Robert Toombs Avenue, Washington",ChIJPyv5gYyv94gR_k9rrzTwuTg


In [16]:
fastfood_df.drop_duplicates(inplace=True)
fastfood_df

,County_Name,Store_Name,Store_Lat,Store_Long,Store_Address,Store_PlaceID
0,Baldwin County,Ruby Tuesday,33.110224,-83.256479,"2440 North Columbia Street, Milledgeville",ChIJWzuGB3q49ogR7nEV2FMDl84
1,Baldwin County,Papa John's Pizza,33.105102,-83.253156,"1960 North Columbia Street, Milledgeville",ChIJjy9AQHa49ogRu4xOOjIXP_E
2,Baldwin County,Waffle House,33.099574,-83.246011,"1683 North Columbia Street, Milledgeville",ChIJBV7JQovH9ogRjb4IDNZftGM
3,Baldwin County,Lieu's Peking Restaurant,33.112239,-83.255534,"2485 North Columbia Street STE 101, Milledgeville",ChIJ3_c7KHq49ogREL_631kUmZM
14,Baldwin County,Country Buffet,33.061440,-83.220369,"1465 South Jefferson Street, Milledgeville",ChIJdbr8vG_I9ogRmpChEdxry4k
...,...,...,...,...,...,...
428,Wilkes County,Shell,33.737671,-82.749277,"16 Lexington Avenue, Washington",ChIJVxfMDZOv94gRYJSaoejrAB0
429,Wilkes County,Econo Lodge Atlanta Airport East,33.615913,-84.396195,"5060 Frontage Road, Forest Park",ChIJyepr11f89IgRH8S2BCnM2EI
430,Wilkes County,Carrabba's Italian Grill,33.567450,-84.329512,"1887 Mount Zion Road, Morrow",ChIJ58ODazf59IgR-mufOqi_Wgc
431,Wilkes County,Clayton State Dining Services,33.596334,-84.331925,"2000 Clayton State Boulevard, Morrow",ChIJ0wPNLMH-9IgRIDw0BScUIBI


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
fastfood_info = [info_box_template.format(**row) for index, row in fastfood_df.iterrows()]
locations = fastfood_df[["Store_Lat","Store_Long"]]

In [31]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = fastfood_info)

fig.add_layer(marker_layer)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))

In [93]:
output_school_file = "../Datasets/schools_final.csv"
school_df.to_csv(output_school_file, sep=',', index=False)

In [94]:
output_store_file = "../Datasets/stores_final.csv"
store_df.to_csv(output_store_file, sep=',', index=False)

In [33]:
output_grocery_store_file = "../Datasets/grocery_store_final.csv"
grocery_stores_df.to_csv(output_grocery_store_file, sep=',', index=False)

In [35]:
output_dollar_store_file = "../Datasets/dollar_store_final.csv"
dollar_stores_df.to_csv(output_dollar_store_file, sep=',', index=False)

In [36]:
output_fast_food_file = "../Datasets/fast_food_final.csv"
fastfood_df.to_csv(output_fast_food_file, sep=',', index=False)